## ETL
El objetivo principal de este notebook es procesar y enriquecer los conjuntos de datos de propiedades, conectividad, POIs y uso de suelo, para condensar toda la información relevante en un único GeoDataFrame listo para el Análisis Exploratorio de Datos (EDA) y la modelación.

In [35]:
# Librerias:
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.neighbors import BallTree
import os

## Extracción
En esta etapa, procedemos a importar la totalidad de los GeoDataFrames trabajados previamente: las bases de datos de propiedades (Venta y Arriendo), y los conjuntos de datos espaciales de conectividad y puntos de interés (POIs). El propósito es establecer la base para la posterior ingeniería de características que unificará la información.

### Importación DataFrames de Conectividad:


In [36]:
#1. Definimos las rutas de los archivos .gpkg de las paradas.
ruta_metro = os.path.join("DB", "Conectividad y POIs", "Paradas", "paradas_metro.gpkg")
ruta_micro = os.path.join("DB", "Conectividad y POIs", "Paradas", "paradas_micro.gpkg")
ruta_tren = os.path.join("DB", "Conectividad y POIs", "Paradas", "paradas_tren.gpkg")
ruta_bus = os.path.join("DB", "Conectividad y POIs", "Paradas", "paradas_bus.gpkg")

#2. Importamos los GeoDataFrames de las paradas.
gdf_metro = gpd.read_file(ruta_metro)
gdf_micro = gpd.read_file(ruta_micro)
gdf_tren = gpd.read_file(ruta_tren)
gdf_bus = gpd.read_file(ruta_bus)

### Importación DataFrames de Puntos de Interés (POIs):

In [37]:
#1. Definimos las rutas de los archivos .gpkg de los POIs.
ruta_salud = os.path.join("DB", "Conectividad y POIs", "POIs", "GeoDataFrames POIs", "gdf_salud.gpkg")
ruta_educacion = os.path.join("DB", "Conectividad y POIs", "POIs", "GeoDataFrames POIs", "gdf_educacion.gpkg")
ruta_comercio_basico = os.path.join("DB", "Conectividad y POIs", "POIs", "GeoDataFrames POIs", "gdf_comercio_basico.gpkg")
ruta_comercio_retail = os.path.join("DB", "Conectividad y POIs", "POIs", "GeoDataFrames POIs", "gdf_comercio_retail.gpkg")
ruta_restauracion = os.path.join("DB", "Conectividad y POIs", "POIs", "GeoDataFrames POIs", "gdf_restauracion.gpkg")
ruta_cultura_ocio = os.path.join("DB", "Conectividad y POIs", "POIs", "GeoDataFrames POIs", "gdf_cultura_ocio.gpkg")
ruta_servicios_financieros = os.path.join("DB", "Conectividad y POIs", "POIs", "GeoDataFrames POIs", "gdf_servicios_financieros.gpkg")
ruta_servicios_generales = os.path.join("DB", "Conectividad y POIs", "POIs", "GeoDataFrames POIs", "gdf_servicios_generales.gpkg")
ruta_seguridad = os.path.join("DB", "Conectividad y POIs", "POIs", "GeoDataFrames POIs", "gdf_seguridad.gpkg")
ruta_alojamiento = os.path.join("DB", "Conectividad y POIs", "POIs", "GeoDataFrames POIs", "gdf_alojamiento.gpkg")

#2. Importamos los GeoDataFrames de los POIs.
gdf_salud = gpd.read_file(ruta_salud)
gdf_educacion = gpd.read_file(ruta_educacion)
gdf_comercio_basico = gpd.read_file(ruta_comercio_basico)
gdf_comercio_retail = gpd.read_file(ruta_comercio_retail)
gdf_restauracion = gpd.read_file(ruta_restauracion)
gdf_cultura_ocio = gpd.read_file(ruta_cultura_ocio)
gdf_servicios_financieros = gpd.read_file(ruta_servicios_financieros)
gdf_servicios_generales = gpd.read_file(ruta_servicios_generales)
gdf_seguridad = gpd.read_file(ruta_seguridad)
gdf_alojamiento = gpd.read_file(ruta_alojamiento)

### Importación DataFrames de Uso de Suelo:

In [38]:
#1. Definimos las rutas de los archivos .gpkg de uso de suelo.
ruta_zona_verde_recreacion = os.path.join("DB", "Conectividad y POIs", "POIs", "GeoDataFrames Suelo", "gdf_zona_verde_recreacion.gpkg")
ruta_zona_comercial_retail = os.path.join("DB", "Conectividad y POIs", "POIs", "GeoDataFrames Suelo", "gdf_zona_comercial_retail.gpkg")
ruta_zona_residencial = os.path.join("DB", "Conectividad y POIs", "POIs", "GeoDataFrames Suelo", "gdf_zona_residencial.gpkg")
ruta_zona_industrial = os.path.join("DB", "Conectividad y POIs", "POIs", "GeoDataFrames Suelo", "gdf_zona_industrial.gpkg")
ruta_zona_agricola = os.path.join("DB", "Conectividad y POIs", "POIs", "GeoDataFrames Suelo", "gdf_zona_agricola.gpkg")
ruta_zona_servicios_especiales = os.path.join("DB", "Conectividad y POIs", "POIs", "GeoDataFrames Suelo", "gdf_zona_servicios_especiales.gpkg")

#2. Importamos los GeoDataFrames de uso de suelo.
gdf_zona_verde_recreacion = gpd.read_file(ruta_zona_verde_recreacion)
gdf_zona_comercial_retail = gpd.read_file(ruta_zona_comercial_retail)
gdf_zona_residencial = gpd.read_file(ruta_zona_residencial)
gdf_zona_industrial = gpd.read_file(ruta_zona_industrial)
gdf_zona_agricola = gpd.read_file(ruta_zona_agricola)
gdf_zona_servicios_especiales = gpd.read_file(ruta_zona_servicios_especiales)

### Importación DataFrames de Propiedades:

In [39]:
#1. Definimos las rutas de los archivos .gpkg de las propiedades.
ruta_venta = os.path.join("DB", "Propiedades", "venta.gpkg")
ruta_arriendo = os.path.join("DB", "Propiedades", "arriendo.gpkg")

#2. Importamos los GeoDataFrames de las propiedades.
gdf_venta = gpd.read_file(ruta_venta)
gdf_arriendo = gpd.read_file(ruta_arriendo)

## Transformación
En primer lugar, se aplica un proceso de limpieza exhaustiva para asegurar la calidad de las geometrías (eliminando puntos nulos o vacíos que impiden el análisis espacial). Posteriormente, se calculan las métricas clave de accesibilidad y conectividad:

* Conteo de Densidad: Se mide el número de POIs y paradas de transporte dentro de un radio de 500 metros para medir la saturación del entorno.
* Proximidad Crítica: Se calcula la distancia al punto más cercano (usando el algoritmo BallTree) y la distancia a las zonas de uso de suelo (polígonos) más próximas.

Este proceso de enriquecimiento dota a cada propiedad de las variables necesarias para modelar su precio y estimar su rentabilidad.

### DataFrames de Propiedades:

In [ ]:
#1. Definimos las columnas de interés en gdf_venta y gdf_arriendo.
columnas_interes = ["inmueble", "region", "comuna", "Dirección", "UF", "$",
       "Superficie total unidad", "Superficie total", "Dormitorios", "Baños",
       "UF/m² zona declarado", "gastos_comunes", "geometry"]

gdf_venta = gdf_venta[columnas_interes]
gdf_arriendo = gdf_arriendo[columnas_interes]

#2. Eliminamos filas con valores nulos en la columna "$".
gdf_venta = gdf_venta.dropna(subset=["$"])
gdf_arriendo = gdf_arriendo.dropna(subset=["$"])

#3. Eliminamos filas con geometrías nulas.
gdf_venta = gdf_venta[~gdf_venta["geometry"].is_empty]
gdf_arriendo = gdf_arriendo[~gdf_arriendo["geometry"].is_empty]

#4. Imputamos valores nulos en todas las columnas con 0.
gdf_venta = gdf_venta.fillna(0)
gdf_arriendo = gdf_arriendo.fillna(0)

#5. Normalizamos los precios de arriendo y venta utilizando una escala logarítmica para reducir el impacto de valores atípicos.
gdf_venta["$_log"] = np.log1p(gdf_venta["$"])
gdf_arriendo["$_log"] = np.log1p(gdf_arriendo["$"])

gdf_arriendo.head()

,inmueble,region,comuna,Dirección,UF,$,Superficie total unidad,Superficie total,Dormitorios,Baños,UF/m² zona declarado,gastos_comunes,geometry,$_log
0,casa,metropolitana,santiago,"Conferencia 1257, Santiago, Club Hípico, Santi...",0.0,440000.0,m²,54.0,4.0,1.0,25.0,0.0,POINT (-70.67482 -33.4648),12.994532
1,casa,metropolitana,santiago,"Barrio San Borja, Santiago, RM (Metropolitana)",0.0,650000.0,m²,700.0,3.0,2.0,33.0,0.0,POINT (-70.64065 -33.44135),13.384729
2,casa,metropolitana,santiago,"Ramón Ángel Jara, Barrio Yungay, Santiago, RM ...",0.0,2000000.0,m²,220.0,5.0,5.0,0.0,0.0,POINT (-70.66986 -33.44019),14.508658
3,casa,metropolitana,santiago,"Compañia // Hurtado Rodriguez, Barrio Yungay, ...",0.0,2700000.0,m²,189.0,10.0,6.0,17.0,0.0,POINT (-70.67524 -33.44132),14.808763
4,casa,metropolitana,santiago,"Rogelio Ugarte 1435, Santiago, Bogotá - Sierra...",0.0,1000000.0,m²,175.0,3.0,1.0,32.0,0.0,POINT (-70.63374 -33.46247),13.815512


### DataFrame de POIs:
Para poder condensar toda la información relevante en un solo GeoDataFrame, es necesario establecer una relación entre la ubicación de las propiedades y los demás puntos de interés. Para ello, definimos una función capaz de contar cuántos POIs hay alrededor de un punto en particular (en este caso, una propiedad) a una distancia de radio m.

In [42]:
def contador_en_radio(gdf_propiedades, gdf_puntos, radio_m, tipo_poi):

    #1. Creamos una copia del GeoDataFrame de propiedades para evitar modificar el original.
    gdf_propiedades = gdf_propiedades.copy()

    #2. Asignamos un nombre temporal al índice para agrupar correctamente.
    gdf_propiedades["propiedad_id"] = gdf_propiedades.index.to_series()
    
    #3. Reproyectamos a sistema métrico.
    proyeccion_propiedades = gdf_propiedades.to_crs("EPSG:32719")
    proyeccion_puntos = gdf_puntos.to_crs("EPSG:32719")

    #4. Creamos un Buffer (círculo) de 500m alrededor de cada propiedad.
    proyeccion_propiedades["geometry"] = proyeccion_propiedades.geometry.buffer(radio_m)

    #5. Unimos las propiedades con los POIs contenidos en el buffer.
    joined = gpd.sjoin(proyeccion_propiedades, proyeccion_puntos, how = "left", predicate = "contains")

    #6. Agrupamos por el índice temporal de las propiedades para contar los POIs.
    count_series = joined.groupby("propiedad_id")["index_right"].count()

    #7. Renombramos la serie y la asignamos de vuelta al GDF original.
    count_col_name = f"puntos_{tipo_poi}_{radio_m}m"
    gdf_propiedades[count_col_name] = count_series
    
    #8. Eliminamos la columna temporal.
    gdf_propiedades.drop(columns=["propiedad_id"], inplace = True)
    
    return gdf_propiedades

A modo de ejemplo, agreguemos al GeoDataFrame de ventas el número de puntos de restauración que hay a 500m de cada propiedad.

In [43]:
gdf_ventas_con_restauracion = contador_en_radio(gdf_venta, gdf_restauracion, 500, "restauracion")
gdf_ventas_con_restauracion.sample(5)

,inmueble,region,comuna,Dirección,UF,$,Superficie total unidad,Superficie total,Dormitorios,Baños,UF/m² zona declarado,gastos_comunes,geometry,$_log,puntos_restauracion_500m
70333,departamento,metropolitana,la-granja,"Sofía Carmona 300 - 600, La Granja Sur, La Gra...",2100.0,8.295868e+07,m²,57.0,3.0,1.0,33.0,40000.0,POINT (-70.629 -33.55803),18.233853,1
32306,departamento,metropolitana,las-condes,"Av. Las Condes 12780, Mall Sport, Las Condes, ...",8500.0,3.367920e+08,m²,102.0,3.0,2.0,99.0,220000.0,POINT (-70.51334 -33.37193),19.634976,14
28529,casa,metropolitana,las-condes,"El Amancay, Quinchamalí, Las Condes, RM (Metro...",26000.0,1.026341e+09,m²,1277.0,5.0,6.0,89.0,0.0,POINT (-70.49636 -33.37073),20.749265,1
101092,casa,metropolitana,lampa,"Chicauma, Lampa, RM (Metropolitana)",4990.0,1.960095e+08,m²,175.0,3.0,3.0,46.0,0.0,POINT (-70.74116 -33.30603),19.093674,0
40944,departamento,metropolitana,vitacura,"Fernando De Arguello, Vitacura, Villa El Dorad...",9200.0,3.645278e+08,m²,90.0,2.0,2.0,117.0,220000.0,POINT (-70.55251 -33.38975),19.714113,4


### DataFrame de Conectividad:
Para el caso del transporte público, la accesibilidad se mide mediante dos factores complementarios: la densidad de paradas en un radio de 500 metros, y la proximidad crítica a la más cercana. Por lo tanto, se define una función que permite determinar la distancia mínima (en kilómetros) de cada propiedad a la parada más próxima, segmentada por el tipo de transporte (metro, micro, tren o bus).

In [44]:
def calcular_distancia_a_punto(gdf_propiedades, gdf_paradas, tipo_transporte):

    #1. Creamos una copia del GeoDataFrame de propiedades para evitar modificar el original.
    gdf_propiedades = gdf_propiedades.copy()
    
    #2. Reproyectamos a sistema métrico.
    proyeccion_propiedades = gdf_propiedades.to_crs("EPSG:32719")
    proyeccion_paradas = gdf_paradas.to_crs("EPSG:32719")

    #3. Extraemos coordenadas de las propiedades y paradas.
    coordenadas_propiedades = np.vstack(proyeccion_propiedades.geometry.apply(lambda geom: (geom.x, geom.y)))
    coordenadas_paradas = np.vstack(proyeccion_paradas.geometry.apply(lambda geom: (geom.x, geom.y)))

    #4. Creamos el BallTree con las coordenadas de las paradas.
    tree = BallTree(coordenadas_paradas)

    #5. Consultamos el BallTree: distancia (dist) e índice (ind) del vecino más cercano (k = 1).
    dist, ind = tree.query(coordenadas_propiedades, k = 1)

    #6. Asignamoas la nueva columna de distancia (convertida a kilómetros).
    dist_tipo_transporte = f"{tipo_transporte}_mas_cercano_km"
    gdf_propiedades[dist_tipo_transporte] = dist.flatten() / 1000

    return gdf_propiedades

A modo de ejemplo, agreguemos al GeoDataFrame de ventas la distancia a la que se encuentra la parada de metro más cercana (en km) por cada propiedad.

In [45]:
gdf_venta_metro = calcular_distancia_a_punto(gdf_venta, gdf_metro, "metro")
gdf_venta_metro.sample(5)

,inmueble,region,comuna,Dirección,UF,$,Superficie total unidad,Superficie total,Dormitorios,Baños,UF/m² zona declarado,gastos_comunes,geometry,$_log,metro_mas_cercano_km
43401,departamento,metropolitana,vitacura,"Pío Xi 900 - 1200, Vitacura, Chile, Pío XI, Vi...",17250.0,6.777630e+08,m²,182.16,4.0,4.0,95.0,332150.0,POINT (-70.58166 -33.39957),20.334308,1.585566
85890,casa,metropolitana,colina,"Chicureo, Chicureo, Colina, RM (Metropolitana)",29600.0,1.166468e+09,m²,5000.00,5.0,3.0,70.0,0.0,POINT (-70.6633 -33.25998),20.877246,11.984279
9957,departamento,metropolitana,santiago,"Sta Victoria 411, Santiago, Santa Isabel, Sant...",0.0,9.100000e+07,m²,48.00,2.0,2.0,74.0,75000.0,POINT (-70.64008 -33.44808),18.326370,0.761646
61728,departamento,metropolitana,huechuraba,"Avenida El Sauce 1 - 300, Pedro Fontova, Huech...",4600.0,1.817190e+08,m²,58.00,2.0,2.0,64.0,150000.0,POINT (-70.68005 -33.35977),19.017972,1.179752
11808,departamento,metropolitana,santiago,"Metro Avenida Matta, San Diego, Santiago, RM (...",3150.0,1.237407e+08,m²,54.00,2.0,2.0,74.0,80000.0,POINT (-70.64414 -33.45837),18.633699,0.116658


### DataFrame de Uso de Suelo:

Para el caso del uso de suelo, la calidad del entorno se mide exclusivamente mediante la proximidad crítica a la zona más influyente. Por lo tanto, se define una función que permite determinar la distancia mínima (en kilómetros) de cada propiedad a la frontera de la zona de uso más cercana, segmentada por el tipo de entorno (industrial, área verde, comercial, residencial, entre otros). 

In [46]:
def calcular_distancia_a_zona(gdf_propiedades, gdf_zonas, tipo_zona):
    
    #1. Creamos una copia del GeoDataFrame de propiedades para evitar modificar el original.
    gdf_propiedades = gdf_propiedades.copy()
    
    #2. Reproyectamos a sistema métrico.
    proyeccion_propiedades = gdf_propiedades.to_crs('EPSG:32719')
    proyeccion_zonas = gdf_zonas.to_crs('EPSG:32719')

    #2. Calculamos la distancia de cada punto al polígono más cercano.
    puntos_propiedades = proyeccion_propiedades.geometry
    distancias = puntos_propiedades.apply(lambda p: proyeccion_zonas.distance(p).min())

    #3. Convertimos a km y asignamos la nueva columna.
    nombre_col = f'{tipo_zona}_mas_cercana_km'
    gdf_propiedades[nombre_col] = distancias / 1000
    
    return gdf_propiedades

## Guardado

Debido a que los datos de conectividad y POIs se encuentran disponibles exclusivamente para la Región Metropolitana, la etapa final de preparación consiste en filtrar y conservar únicamente las propiedades ubicadas dentro de esta región. Este paso metodológico garantiza que cada fila del GeoDataFrame unificado posea la información completa de accesibilidad y entorno, permitiendo llevar a cabo un análisis multivariable robusto y consistente.

### Filtro para propiedades dentro de la Región Metropolitana:

In [47]:
#1. Filtramos el DataFrame de venta.
gdf_venta_metropolitana = gdf_venta[gdf_venta["region"] == "metropolitana"]

#2. Filtramos el DataFrame de arriendo.
gdf_arriendo_metropolitana = gdf_arriendo[gdf_arriendo["region"] == "metropolitana"]

### GeoDataFrame de Venta:

* Contador de Paradas

In [53]:
#1. Paradas de Metro
gdf_venta_metropolitana = contador_en_radio(gdf_venta_metropolitana, gdf_metro, 500, "metro")
#2. Paradas de Micro
gdf_venta_metropolitana = contador_en_radio(gdf_venta_metropolitana, gdf_micro, 500, "micro")
#3. Paradas de Tren
gdf_venta_metropolitana = contador_en_radio(gdf_venta_metropolitana, gdf_tren, 500, "tren")
#4. Paradas de Bus
gdf_venta_metropolitana = contador_en_radio(gdf_venta_metropolitana, gdf_bus, 500, "bus")

* Parada más cercana

In [54]:
#1. Metro más cercano
gdf_venta_metropolitana = calcular_distancia_a_punto(gdf_venta_metropolitana, gdf_metro, "metro")
#2. Micro más cercano
gdf_venta_metropolitana = calcular_distancia_a_punto(gdf_venta_metropolitana, gdf_micro, "micro")
#3. Tren más cercano
gdf_venta_metropolitana = calcular_distancia_a_punto(gdf_venta_metropolitana, gdf_tren, "tren")
#4. Bus más cercano
gdf_venta_metropolitana = calcular_distancia_a_punto(gdf_venta_metropolitana, gdf_bus, "bus")

* Contador de POIs

In [55]:
#1. Salud
gdf_venta_metropolitana = contador_en_radio(gdf_venta_metropolitana, gdf_salud, 500, "salud")
#2. Educación
gdf_venta_metropolitana = contador_en_radio(gdf_venta_metropolitana, gdf_educacion, 500, "educacion")
#3. Comercio Básico
gdf_venta_metropolitana = contador_en_radio(gdf_venta_metropolitana, gdf_comercio_basico, 500, "comercio_basico")
#4. Comercio Retail
gdf_venta_metropolitana = contador_en_radio(gdf_venta_metropolitana, gdf_comercio_retail, 500, "comercio_retail")
#5. Restauración
gdf_venta_metropolitana = contador_en_radio(gdf_venta_metropolitana, gdf_restauracion, 500, "restauracion")
#6. Cultura y Ocio
gdf_venta_metropolitana = contador_en_radio(gdf_venta_metropolitana, gdf_cultura_ocio, 500, "cultura_ocio")
#7. Servicios Financieros
gdf_venta_metropolitana = contador_en_radio(gdf_venta_metropolitana, gdf_servicios_financieros, 500, "servicios_financieros")
#8. Servicios Generales
gdf_venta_metropolitana = contador_en_radio(gdf_venta_metropolitana, gdf_servicios_generales, 500, "servicios_generales")
#9. Seguridad
gdf_venta_metropolitana = contador_en_radio(gdf_venta_metropolitana, gdf_seguridad, 500, "seguridad")
#10. Alojamiento
gdf_venta_metropolitana = contador_en_radio(gdf_venta_metropolitana, gdf_alojamiento, 500, "alojamiento")

* POIs más cercanos

In [57]:
#1. Salud más cercano
gdf_venta_metropolitana = calcular_distancia_a_punto(gdf_venta_metropolitana, gdf_salud, "salud")
#2. Educación más cercano
gdf_venta_metropolitana = calcular_distancia_a_punto(gdf_venta_metropolitana, gdf_educacion, "educacion")
#3. Comercio Básico más cercano
gdf_venta_metropolitana = calcular_distancia_a_punto(gdf_venta_metropolitana, gdf_comercio_basico, "comercio_basico")
#4. Comercio Retail más cercano
gdf_venta_metropolitana = calcular_distancia_a_punto(gdf_venta_metropolitana, gdf_comercio_retail, "comercio_retail")
#5. Restauración más cercano
gdf_venta_metropolitana = calcular_distancia_a_punto(gdf_venta_metropolitana, gdf_restauracion, "restauracion")
#6. Cultura y Ocio más cercano
gdf_venta_metropolitana = calcular_distancia_a_punto(gdf_venta_metropolitana, gdf_cultura_ocio, "cultura_ocio")
#7. Servicios Financieros más cercano
gdf_venta_metropolitana = calcular_distancia_a_punto(gdf_venta_metropolitana, gdf_servicios_financieros, "servicios_financieros")
#8. Servicios Generales más cercano
gdf_venta_metropolitana = calcular_distancia_a_punto(gdf_venta_metropolitana, gdf_servicios_generales, "servicios_generales")
#9. Seguridad más cercano
gdf_venta_metropolitana = calcular_distancia_a_punto(gdf_venta_metropolitana, gdf_seguridad, "seguridad")
#10. Alojamiento más cercano
gdf_venta_metropolitana = calcular_distancia_a_punto(gdf_venta_metropolitana, gdf_alojamiento, "alojamiento")

* Zonas más cercanas

In [58]:
#1. Zona Verde y Recreación
gdf_venta_metropolitana = calcular_distancia_a_zona(gdf_venta_metropolitana, gdf_zona_verde_recreacion, "zona_verde_recreacion")
#2. Zona Comercial Retail
gdf_venta_metropolitana = calcular_distancia_a_zona(gdf_venta_metropolitana, gdf_zona_comercial_retail, "zona_comercial_retail")
#3. Zona Residencial
gdf_venta_metropolitana = calcular_distancia_a_zona(gdf_venta_metropolitana, gdf_zona_residencial, "zona_residencial")
#4. Zona Industrial
gdf_venta_metropolitana = calcular_distancia_a_zona(gdf_venta_metropolitana, gdf_zona_industrial, "zona_industrial")
#5. Zona Agrícola
gdf_venta_metropolitana = calcular_distancia_a_zona(gdf_venta_metropolitana, gdf_zona_agricola, "zona_agricola")
#6. Zona Servicios Especiales
gdf_venta_metropolitana = calcular_distancia_a_zona(gdf_venta_metropolitana, gdf_zona_servicios_especiales, "zona_servicios_especiales")

### GeoDataFrame de Arriendo:

* Contador de Paradas

In [59]:
#1. Paradas de Metro
gdf_arriendo_metropolitana = contador_en_radio(gdf_arriendo_metropolitana, gdf_metro, 500, "metro")
#2. Paradas de Micro
gdf_arriendo_metropolitana = contador_en_radio(gdf_arriendo_metropolitana, gdf_micro, 500, "micro")
#3. Paradas de Tren
gdf_arriendo_metropolitana = contador_en_radio(gdf_arriendo_metropolitana, gdf_tren, 500, "tren")
#4. Paradas de Bus
gdf_arriendo_metropolitana = contador_en_radio(gdf_arriendo_metropolitana, gdf_bus, 500, "bus")

* Parada más cercana

In [60]:
#1. Metro más cercano
gdf_arriendo_metropolitana = calcular_distancia_a_punto(gdf_arriendo_metropolitana, gdf_metro, "metro")
#2. Micro más cercano
gdf_arriendo_metropolitana = calcular_distancia_a_punto(gdf_arriendo_metropolitana, gdf_micro, "micro")
#3. Tren más cercano
gdf_arriendo_metropolitana = calcular_distancia_a_punto(gdf_arriendo_metropolitana, gdf_tren, "tren")
#4. Bus más cercano
gdf_arriendo_metropolitana = calcular_distancia_a_punto(gdf_arriendo_metropolitana, gdf_bus, "bus")

* Contador de POIs

In [61]:
#1. Salud
gdf_arriendo_metropolitana = contador_en_radio(gdf_arriendo_metropolitana, gdf_salud, 500, "salud")
#2. Educación
gdf_arriendo_metropolitana = contador_en_radio(gdf_arriendo_metropolitana, gdf_educacion, 500, "educacion")
#3. Comercio Básico
gdf_arriendo_metropolitana = contador_en_radio(gdf_arriendo_metropolitana, gdf_comercio_basico, 500, "comercio_basico")
#4. Comercio Retail
gdf_arriendo_metropolitana = contador_en_radio(gdf_arriendo_metropolitana, gdf_comercio_retail, 500, "comercio_retail")
#5. Restauración
gdf_arriendo_metropolitana = contador_en_radio(gdf_arriendo_metropolitana, gdf_restauracion, 500, "restauracion")
#6. Cultura y Ocio
gdf_arriendo_metropolitana = contador_en_radio(gdf_arriendo_metropolitana, gdf_cultura_ocio, 500, "cultura_ocio")
#7. Servicios Financieros
gdf_arriendo_metropolitana = contador_en_radio(gdf_arriendo_metropolitana, gdf_servicios_financieros, 500, "servicios_financieros")
#8. Servicios Generales
gdf_arriendo_metropolitana = contador_en_radio(gdf_arriendo_metropolitana, gdf_servicios_generales, 500, "servicios_generales")
#9. Seguridad
gdf_arriendo_metropolitana = contador_en_radio(gdf_arriendo_metropolitana, gdf_seguridad, 500, "seguridad")
#10. Alojamiento
gdf_arriendo_metropolitana = contador_en_radio(gdf_arriendo_metropolitana, gdf_alojamiento, 500, "alojamiento")

* POIs más cercanos

In [62]:
#1. Salud más cercano
gdf_arriendo_metropolitana = calcular_distancia_a_punto(gdf_arriendo_metropolitana, gdf_salud, "salud")
#2. Educación más cercano
gdf_arriendo_metropolitana = calcular_distancia_a_punto(gdf_arriendo_metropolitana, gdf_educacion, "educacion")
#3. Comercio Básico más cercano
gdf_arriendo_metropolitana = calcular_distancia_a_punto(gdf_arriendo_metropolitana, gdf_comercio_basico, "comercio_basico")
#4. Comercio Retail más cercano
gdf_arriendo_metropolitana = calcular_distancia_a_punto(gdf_arriendo_metropolitana, gdf_comercio_retail, "comercio_retail")
#5. Restauración más cercano
gdf_arriendo_metropolitana = calcular_distancia_a_punto(gdf_arriendo_metropolitana, gdf_restauracion, "restauracion")
#6. Cultura y Ocio más cercano
gdf_arriendo_metropolitana = calcular_distancia_a_punto(gdf_arriendo_metropolitana, gdf_cultura_ocio, "cultura_ocio")
#7. Servicios Financieros más cercano
gdf_arriendo_metropolitana = calcular_distancia_a_punto(gdf_arriendo_metropolitana, gdf_servicios_financieros, "servicios_financieros")
#8. Servicios Generales más cercano
gdf_arriendo_metropolitana = calcular_distancia_a_punto(gdf_arriendo_metropolitana, gdf_servicios_generales, "servicios_generales")
#9. Seguridad más cercano
gdf_arriendo_metropolitana = calcular_distancia_a_punto(gdf_arriendo_metropolitana, gdf_seguridad, "seguridad")
#10. Alojamiento más cercano
gdf_arriendo_metropolitana = calcular_distancia_a_punto(gdf_arriendo_metropolitana, gdf_alojamiento, "alojamiento")

* Zonas más cercanas

In [63]:
#1. Zona Verde y Recreación
gdf_arriendo_metropolitana = calcular_distancia_a_zona(gdf_arriendo_metropolitana, gdf_zona_verde_recreacion, "zona_verde_recreacion")
#2. Zona Comercial Retail
gdf_arriendo_metropolitana = calcular_distancia_a_zona(gdf_arriendo_metropolitana, gdf_zona_comercial_retail, "zona_comercial_retail")
#3. Zona Residencial
gdf_arriendo_metropolitana = calcular_distancia_a_zona(gdf_arriendo_metropolitana, gdf_zona_residencial, "zona_residencial")
#4. Zona Industrial
gdf_arriendo_metropolitana = calcular_distancia_a_zona(gdf_arriendo_metropolitana, gdf_zona_industrial, "zona_industrial")
#5. Zona Agrícola
gdf_arriendo_metropolitana = calcular_distancia_a_zona(gdf_arriendo_metropolitana, gdf_zona_agricola, "zona_agricola")
#6. Zona Servicios Especiales
gdf_arriendo_metropolitana = calcular_distancia_a_zona(gdf_arriendo_metropolitana, gdf_zona_servicios_especiales, "zona_servicios_especiales")

In [65]:
gdf_venta_metropolitana.columns

Index(['inmueble', 'region', 'comuna', 'Dirección', 'UF', '$',
       'Superficie total unidad', 'Superficie total', 'Dormitorios', 'Baños',
       'UF/m² zona declarado', 'gastos_comunes', 'geometry', '$_log',
       'puntos_tren_500m', 'puntos_bus_500m', 'puntos_metro_500m',
       'puntos_micro_500m', 'metro_mas_cercano_km', 'micro_mas_cercano_km',
       'tren_mas_cercano_km', 'bus_mas_cercano_km', 'puntos_salud_500m',
       'puntos_educacion_500m', 'puntos_comercio_basico_500m',
       'puntos_comercio_retail_500m', 'puntos_restauracion_500m',
       'puntos_cultura_ocio_500m', 'puntos_servicios_financieros_500m',
       'puntos_servicios_generales_500m', 'puntos_seguridad_500m',
       'puntos_alojamiento_500m', 'salud_mas_cercano_km',
       'educacion_mas_cercano_km', 'comercio_basico_mas_cercano_km',
       'comercio_retail_mas_cercano_km', 'restauracion_mas_cercano_km',
       'cultura_ocio_mas_cercano_km', 'servicios_financieros_mas_cercano_km',
       'servicios_generales_

### Exportación a .gpkg y .csv:

In [70]:
#1. Reorganizamos las columnas del GeoDataFrame.
columnas_ordenadas = [
    # Columnas originales
    "Dirección", "inmueble", "region", "comuna", "UF", "$", "$_log",
    "Superficie total unidad", "Superficie total", "Dormitorios", "Baños",
    "UF/m² zona declarado", "gastos_comunes", "geometry",
    # Columnas de conectividad
    "puntos_metro_500m", "metro_mas_cercano_km",
    "puntos_micro_500m", "micro_mas_cercano_km",
    "puntos_tren_500m", "tren_mas_cercano_km",
    "puntos_bus_500m", "bus_mas_cercano_km",
    # Columnas de POIs
    "puntos_salud_500m", "salud_mas_cercano_km",
    "puntos_educacion_500m", "educacion_mas_cercano_km",
    "puntos_comercio_basico_500m", "comercio_basico_mas_cercano_km",
    "puntos_comercio_retail_500m", "comercio_retail_mas_cercano_km",
    "puntos_restauracion_500m", "restauracion_mas_cercano_km",
    "puntos_cultura_ocio_500m", "cultura_ocio_mas_cercano_km",
    "puntos_servicios_financieros_500m", "servicios_financieros_mas_cercano_km",
    "puntos_servicios_generales_500m", "servicios_generales_mas_cercano_km",
    "puntos_seguridad_500m", "seguridad_mas_cercano_km",
    "puntos_alojamiento_500m", "alojamiento_mas_cercano_km",
    # Columnas de Zonas
    "zona_verde_recreacion_mas_cercana_km",
    "zona_comercial_retail_mas_cercana_km",
    "zona_residencial_mas_cercana_km",
    "zona_industrial_mas_cercana_km",
    "zona_agricola_mas_cercana_km",
    "zona_servicios_especiales_mas_cercana_km"
]

#2. Aplicamos el reordenamiento.
gdf_venta_metropolitana = gdf_venta_metropolitana[columnas_ordenadas]
gdf_arriendo_metropolitana = gdf_arriendo_metropolitana[columnas_ordenadas]

#3. Guardamos los GeoDataFrames enriquecidos en archivos .gpkg.
ruta_salida_venta = os.path.join("DB", "Propiedades", "GeoDataFrames Procesados (EDA)", "gdf_venta_metropolitana.gpkg")
ruta_salida_arriendo = os.path.join("DB", "Propiedades", "GeoDataFrames Procesados (EDA)", "gdf_arriendo_metropolitana.gpkg")

gdf_venta_metropolitana.to_file(ruta_salida_venta, driver = "GPKG")
gdf_arriendo_metropolitana.to_file(ruta_salida_arriendo, driver = "GPKG")

#4. Guardamos los GeoDataFrames enriquecidos en archivos .csv.
ruta_salida_venta_csv = os.path.join("DB", "Propiedades", "GeoDataFrames Procesados (EDA)", "gdf_venta_metropolitana.csv")
ruta_salida_arriendo_csv = os.path.join("DB", "Propiedades", "GeoDataFrames Procesados (EDA)", "gdf_arriendo_metropolitana.csv")

gdf_venta_metropolitana.to_csv(ruta_salida_venta_csv, index = False)
gdf_arriendo_metropolitana.to_csv(ruta_salida_arriendo_csv, index = False)